# Import openai library

In [1]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00


In [5]:
!pip install -q multiprocess timeout_decorator pebble

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00


In [3]:
!gdown 1T-OGrjWLdDeaUBGDCA14wghW06vzU7KN

Downloading...
From (original): https://drive.google.com/uc?id=1T-OGrjWLdDeaUBGDCA14wghW06vzU7KN
From (redirected): https://drive.google.com/uc?id=1T-OGrjWLdDeaUBGDCA14wghW06vzU7KN&confirm=t&uuid=a8150a70-ebeb-404c-bf75-2c0db60bc6e8
To: /content/python_executor.py
100% 6.31k/6.31k [00:00<00:00, 23.0MB/s]


# Init conversation

In [4]:
from openai import OpenAI
import os


def get_completion(prompt, params):
    messages = [{"role": "user", "content":prompt}]
    client = OpenAI(
      api_key=os.environ.get("OPENAI_API_KEY"),
    )
    response = client.chat.completions.create(
        **params,
        messages=messages
    )

    return response.choices[0].message.content


os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY"

modelName = 'gpt-3.5-turbo'
temperature = 0.7
top_p = 1

params = {
    "model": modelName,
    "temperature": temperature,
    "top_p": top_p
}

In [ ]:
# Without Synthetic-prompting
prompt = '''
Q: Linh mua 1 túi gạo giá $15, Linh có $14 và 150 cent. Hỏi Linh còn lại bao nhiêu tiền
A:
'''
response = get_completion(prompt, params)
print(response)

Để trả được túi gạo giá $15, Linh cần tổng cộng 1500 cent (15x100). Linh đã có $14, tương đương với 1400 cent. Vậy Linh còn thiếu 100 cent nữa để đủ trả cho túi gạo. 

Vậy nên Linh còn lại $14 và 50 cent.


In [9]:
# Synthetic-prompting
prompt = '''
Ví dụ 1:
Viết một hàm tên là solution() để giải quyết vấn đề được nêu ở dưới
Chủ đề nói về việc làm
def solution():
    # 1
    num_roses = 4
    # 2
    num_dahlias = num_roses + 7
    # 3
    num_flowers = num_roses + num_dahlias
    # 4
    result = num_flowers
    # 5
    return result
Q: Có 4 bông hoa hồng. Số hoa cúc nhiều hơn số hoa hồng 7 bông. Có bao nhiêu bông hoa tất cả?
Ví dụ 2:
Viết một hàm tên là solution() để giải quyết vấn đề được nêu ở dưới
Chủ đề nói về nấu ăn
def solution():
    # 1
    bag_of_onions = 4
    # 2
    weights_per_bag = 50
    # 3
    cost_per_bag = weights_per_bag * 1.5
    # 4
    money_spend = bag_of_onions * cost_per_bag
    # 5
    result = money_spend
    # 6
    return result
Q: Một đầu bếp mua 4 túi hành. Mỗi túi nặng 50 pounds. Mỗi pound tốn $1.50. Hỏi đầu bếp phải chi trả bao nhiêu?
Ví dụ 3:
Q: Lan ăn 1 cây kem vào mỗi buổi tối. Biết 1 hộp kem giá $4.00 gồm có 15 cây kem. Hỏi Lan phải chi trả bao nhiêu tiền để mua kem trong vòng 60 ngày.
A:
def solution():
    servings_per_night = 1
    servings_per_carton = 15
    cost_per_carton = 4.00
    num_days = 60
    num_servings = num_days * servings_per_night
    num_cartons = num_servings / servings_per_carton
    money_spent = cost_per_carton * num_cartons
    result = money_spent
    return result
print(solution())

Q: Linh mua 1 túi gạo giá $15, Linh có $14 và 150 cent. Hỏi Linh còn lại bao nhiêu tiền
A:
'''
response = get_completion(prompt, params)
print(response)

from python_executor import PythonExecutor
response = get_completion(prompt, params)
executor = PythonExecutor(get_answer_from_stdout=True)
predictions = executor.apply(response)
print('Đáp án của câu hỏi là: ' + predictions[0])

final_prompt = f'''
Q: Linh mua 1 túi gạo giá $15, Linh có $14 và 150 cent. Hỏi Linh còn lại bao nhiêu tiền. Cho biết đáp án là {predictions[0]}
A:
'''
response = get_completion(final_prompt, params)
print('-------------------------------------------------------')
print(response)

def solution():
    cost_of_rice = 15.00
    money_Linh_has = 14.00 + 1.50
    remaining_money = money_Linh_has - cost_of_rice
    return remaining_money

print(solution())
Đáp án của câu hỏi là: 0.5
-------------------------------------------------------
Để tính số tiền Linh còn lại sau khi mua túi gạo, ta cần chuyển đổi số tiền Linh có từ cent sang đồng.
- $14 = 1400 cent
- Tổng số tiền Linh có là 1400 cent + 150 cent = 1550 cent
- Giá của túi gạo là $15 = 1500 cent
- Số tiền Linh còn lại sau khi mua túi gạo là 1550 cent - 1500 cent = 50 cent = $0.5

Vậy, Linh còn lại $0.5.
